In [ ]:
# default_exp models.factorization

# Factorization
> Matrix Factorization algorithmic modules for recommender systems.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
#export
class MF(nn.Module):
    def __init__(self, num_users, num_items, emb_size=100):
        super(MF, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.item_emb = nn.Embedding(num_items, emb_size)
        self.user_emb.weight.data.uniform_(0, 0.05)
        self.item_emb.weight.data.uniform_(0, 0.05)
        
    def forward(self, u, v):
        u = self.user_emb(u)
        v = self.item_emb(v)
        return (u*v).sum(1)

In [ ]:
#export
class BiasedMF(nn.Module):
    def __init__(self, num_users, num_items, emb_size=100):
        super(BiasedMF, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.user_bias = nn.Embedding(num_users, 1)
        self.item_emb = nn.Embedding(num_items, emb_size)
        self.item_bias = nn.Embedding(num_items, 1)
        self.user_emb.weight.data.uniform_(0,0.05)
        self.item_emb.weight.data.uniform_(0,0.05)
        self.user_bias.weight.data.uniform_(-0.01,0.01)
        self.item_bias.weight.data.uniform_(-0.01,0.01)
        
    def forward(self, u, v):
        U = self.user_emb(u)
        V = self.item_emb(v)
        b_u = self.user_bias(u).squeeze()
        b_v = self.item_bias(v).squeeze()
        return (U*V).sum(1) +  b_u  + b_v

In [ ]:
#export
class NeuMF(nn.Module):
    def __init__(self, args, num_users, num_items):
        super(NeuMF, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.factor_num_mf = args.factor_num
        self.factor_num_mlp =  int(args.layers[0]/2)
        self.layers = args.layers
        self.dropout = args.dropout

        self.embedding_user_mlp = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num_mlp)
        self.embedding_item_mlp = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num_mlp)

        self.embedding_user_mf = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num_mf)
        self.embedding_item_mf = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num_mf)

        self.fc_layers = nn.ModuleList()
        for idx, (in_size, out_size) in enumerate(zip(args.layers[:-1], args.layers[1:])):
            self.fc_layers.append(torch.nn.Linear(in_size, out_size))
            self.fc_layers.append(nn.ReLU())

        self.affine_output = nn.Linear(in_features=args.layers[-1] + self.factor_num_mf, out_features=1)
        self.logistic = nn.Sigmoid()
        self.init_weight()

    def init_weight(self):
        nn.init.normal_(self.embedding_user_mlp.weight, std=0.01)
        nn.init.normal_(self.embedding_item_mlp.weight, std=0.01)
        nn.init.normal_(self.embedding_user_mf.weight, std=0.01)
        nn.init.normal_(self.embedding_item_mf.weight, std=0.01)
        
        for m in self.fc_layers:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                
        nn.init.xavier_uniform_(self.affine_output.weight)

        for m in self.modules():
            if isinstance(m, nn.Linear) and m.bias is not None:
                m.bias.data.zero_()

    def forward(self, user_indices, item_indices):
        user_embedding_mlp = self.embedding_user_mlp(user_indices)
        item_embedding_mlp = self.embedding_item_mlp(item_indices)

        user_embedding_mf = self.embedding_user_mf(user_indices)
        item_embedding_mf = self.embedding_item_mf(item_indices)

        mlp_vector = torch.cat([user_embedding_mlp, item_embedding_mlp], dim=-1)
        mf_vector =torch.mul(user_embedding_mf, item_embedding_mf)

        for idx, _ in enumerate(range(len(self.fc_layers))):
            mlp_vector = self.fc_layers[idx](mlp_vector)

        vector = torch.cat([mlp_vector, mf_vector], dim=-1)
        logits = self.affine_output(vector)
        rating = self.logistic(logits)
        return rating.squeeze()

In [ ]:
#export
class GMF(nn.Module):
    def __init__(self, args, num_users, num_items):
        super(GMF, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.factor_num = args.factor_num

        self.embedding_user = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num)
        self.embedding_item = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num)

        self.affine_output = nn.Linear(in_features=self.factor_num, out_features=1)
        self.logistic = nn.Sigmoid()

    def forward(self, user_indices, item_indices):
        user_embedding = self.embedding_user(user_indices)
        item_embedding = self.embedding_item(item_indices)
        element_product = torch.mul(user_embedding, item_embedding)
        logits = self.affine_output(element_product)
        rating = self.logistic(logits)
        return rating

    def init_weight(self):
        pass

In [ ]:
# import torch
# import torch.nn.functional as F

# from recohut.datasets.synthetic import Synthetic
# from recohut.transforms.split import chrono_split
# from recohut.transforms.encode import label_encode as le
# from recohut.models.dnn import CollabFNet

# # generate synthetic implicit data
# synt = Synthetic()
# df = synt.implicit()

# # drop duplicates
# df = df.drop_duplicates()

# # chronological split
# df_train, df_valid = chrono_split(df)
# print(f"Train set:\n\n{df_train}\n{'='*100}\n")
# print(f"Validation set:\n\n{df_valid}\n{'='*100}\n")

# # label encoding
# df_train, uid_maps = le(df_train, col='USERID')
# df_train, iid_maps = le(df_train, col='ITEMID')
# df_valid = le(df_valid, col='USERID', maps=uid_maps)
# df_valid = le(df_valid, col='ITEMID', maps=iid_maps)

# # event implicit to rating conversion
# event_weights = {'click':1, 'add':2, 'purchase':4}
# event_maps = dict({'EVENT_TO_IDX':event_weights})
# df_train = le(df_train, col='EVENT', maps=event_maps)
# df_valid = le(df_valid, col='EVENT', maps=event_maps)
# print(f"Processed Train set:\n\n{df_train}\n{'='*100}\n")
# print(f"Processed Validation set:\n\n{df_valid}\n{'='*100}\n")

# # get number of unique users and items
# num_users = len(df_train.USERID.unique())
# num_items = len(df_train.ITEMID.unique())
# print(f"There are {num_users} users and {num_items} items.\n{'='*100}\n")

# # training and testing related helper functions
# def train_epocs(model, epochs=10, lr=0.01, wd=0.0, unsqueeze=False):
#     optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
#     model.train()
#     for i in range(epochs):
#         users = torch.LongTensor(df_train.USERID.values) # .cuda()
#         items = torch.LongTensor(df_train.ITEMID.values) #.cuda()
#         ratings = torch.FloatTensor(df_train.EVENT.values) #.cuda()
#         if unsqueeze:
#             ratings = ratings.unsqueeze(1)
#         y_hat = model(users, items)
#         loss = F.mse_loss(y_hat, ratings)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         print(loss.item()) 
#     test_loss(model, unsqueeze)

# def test_loss(model, unsqueeze=False):
#     model.eval()
#     users = torch.LongTensor(df_valid.USERID.values) #.cuda()
#     items = torch.LongTensor(df_valid.ITEMID.values) #.cuda()
#     ratings = torch.FloatTensor(df_valid.EVENT.values) #.cuda()
#     if unsqueeze:
#         ratings = ratings.unsqueeze(1)
#     y_hat = model(users, items)
#     loss = F.mse_loss(y_hat, ratings)
#     print("test loss %.3f " % loss.item())

# # training MF model
# model = MF(num_users, num_items, emb_size=100) # .cuda() if you have a GPU
# print(f"Training MF model:\n")
# train_epocs(model, epochs=10, lr=0.1)
# print(f"\n{'='*100}\n")

# # training MF with bias model
# model = MF_bias(num_users, num_items, emb_size=100) #.cuda()
# print(f"Training MF+bias model:\n")
# train_epocs(model, epochs=10, lr=0.05, wd=1e-5)
# print(f"\n{'='*100}\n")

# # training MLP model
# model = CollabFNet(num_users, num_items, emb_size=100) #.cuda()
# print(f"Training MLP model:\n")
# train_epocs(model, epochs=15, lr=0.05, wd=1e-6, unsqueeze=True)
# print(f"\n{'='*100}\n")


# """
# Train set:
#     USERID  ITEMID     EVENT   TIMESTAMP
# 0        1       1     click  2000-01-01
# 2        1       2     click  2000-01-02
# 5        2       1     click  2000-01-01
# 6        2       2  purchase  2000-01-01
# 7        2       1       add  2000-01-03
# 8        2       2  purchase  2000-01-03
# 10       3       3     click  2000-01-01
# 11       3       3     click  2000-01-03
# 12       3       3       add  2000-01-03
# 13       3       3  purchase  2000-01-03
# ====================================================================================================
# Validation set:
#     USERID  ITEMID     EVENT   TIMESTAMP
# 4        1       2  purchase  2000-01-02
# 9        2       3  purchase  2000-01-03
# 14       3       1     click  2000-01-04
# ====================================================================================================
# Processed Train set:
#     USERID  ITEMID  EVENT   TIMESTAMP
# 0        0       0      1  2000-01-01
# 2        0       1      1  2000-01-02
# 5        1       0      1  2000-01-01
# 6        1       1      4  2000-01-01
# 7        1       0      2  2000-01-03
# 8        1       1      4  2000-01-03
# 10       2       2      1  2000-01-01
# 11       2       2      1  2000-01-03
# 12       2       2      2  2000-01-03
# 13       2       2      4  2000-01-03
# ====================================================================================================
# Processed Validation set:
#     USERID  ITEMID  EVENT   TIMESTAMP
# 4        0       1      4  2000-01-02
# 9        1       2      4  2000-01-03
# 14       2       0      1  2000-01-04
# ====================================================================================================
# There are 3 users and 3 items.
# ====================================================================================================
# Training MF model:
# 5.836816787719727
# 1.993103265762329
# 4.549840450286865
# 1.5779536962509155
# 1.285771131515503
# 1.926152229309082
# 2.242276191711426
# 2.270019054412842
# 2.3635096549987793
# 2.272618055343628
# test loss 9.208 
# ====================================================================================================
# Training MF+bias model:
# 5.8399200439453125
# 3.7661311626434326
# 1.8716331720352173
# 1.6015545129776
# 1.5306222438812256
# 1.2995147705078125
# 1.1046849489212036
# 1.1331274509429932
# 1.2109991312026978
# 1.2451963424682617
# test loss 5.625 
# ====================================================================================================
# Training MLP model:
# 4.953568458557129
# 9.649873733520508
# 1.1805670261383057
# 2.54287052154541
# 2.6113314628601074
# 2.1839144229888916
# 1.4144573211669922
# 0.8893814086914062
# 0.7603365182876587
# 1.240354061126709
# 1.1316341161727905
# 0.8014519810676575
# 0.7997692823410034
# 0.8474739789962769
# 0.9691768884658813
# test loss 5.207 
# ====================================================================================================
# """

In [ ]:
#hide
!pip install -q watermark
%reload_ext watermark
%watermark -a "Sparsh A." -m -iv -u -t -d

Author: Sparsh A.

Last updated: 2021-12-18 06:19:22

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.4.104+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

torch  : 1.10.0+cu111
IPython: 5.5.0

